In [1]:
from fbprophet import Prophet

Importing plotly failed. Interactive plots will not work.


In [2]:
df_preCovid_train = pd.read_pickle("../Data/Subway_Data/preCovid_train.pkl")
df_preCovid_test = pd.read_pickle("../Data/Subway_Data/preCovid_test.pkl")

In [3]:
df_preCovid_train.head()

,entries,year,month,day,weekday,timeindex,trend,entries_detrended,entries_detrended_deweekdayed,entries_detrended_deweekdayed_demonthed,factor_model
date,,,,,,,,,,,
2015-01-03,2682846.0,2015,1,3,5,0.0,4.713360e+06,0.569200,0.787814,0.847921,3.164028e+06
2015-01-04,2481666.0,2015,1,4,6,1.0,4.713276e+06,0.526527,0.901450,0.970227,2.557820e+06
2015-01-05,5104111.0,2015,1,5,0,2.0,4.713192e+06,1.082941,1.022742,1.100773,4.636842e+06
2015-01-06,5061648.0,2015,1,6,1,3.0,4.713108e+06,1.073951,0.938890,1.010524,5.008935e+06
2015-01-07,5116883.0,2015,1,7,2,4.0,4.713024e+06,1.085690,0.926845,0.997559,5.129403e+06


In [4]:
# creating a new df for prophet test

preCovid_prophet = df_preCovid_train.copy(deep=True)
preCovid_prophet['ds'] = preCovid_prophet.index
preCovid_prophet['y'] = preCovid_prophet['entries']
preCovid_prophet.reset_index(inplace=True)
preCovid_prophet = preCovid_prophet[['ds', 'y']]

In [5]:
preCovid_prophet.head()

,ds,y
0,2015-01-03,2682846.0
1,2015-01-04,2481666.0
2,2015-01-05,5104111.0
3,2015-01-06,5061648.0
4,2015-01-07,5116883.0


In [6]:
preCovid_prophet.shape

(1459, 2)

In [ ]:
# fitting df to prophet

m = Prophet(yearly_seasonality=False)
m.fit(preCovid_prophet)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
